<a href="https://colab.research.google.com/github/greyhound101/work_in_progress/blob/master/molecule_transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os
os.environ['KAGGLE_USERNAME'] = "" # username from the json file
os.environ['KAGGLE_KEY'] = "" # key from the json file
!kaggle competitions download -c champs-scalar-coupling

  0% 0.00/386 [00:00<?, ?B/s]
100% 386/386 [00:00<00:00, 776kB/s]
  0% 0.00/212 [00:00<?, ?B/s]
100% 212/212 [00:00<00:00, 210kB/s]
  0% 0.00/171 [00:00<?, ?B/s]
100% 171/171 [00:00<00:00, 153kB/s]
  0% 0.00/168 [00:00<?, ?B/s]
100% 168/168 [00:00<00:00, 166kB/s]
  0% 0.00/385 [00:00<?, ?B/s]
100% 385/385 [00:00<00:00, 335kB/s]
  0% 0.00/254 [00:00<?, ?B/s]
100% 254/254 [00:00<00:00, 223kB/s]
  0% 0.00/470 [00:00<?, ?B/s]
100% 470/470 [00:00<00:00, 481kB/s]
  0% 0.00/126 [00:00<?, ?B/s]
100% 126/126 [00:00<00:00, 128kB/s]
  0% 0.00/341 [00:00<?, ?B/s]
100% 341/341 [00:00<00:00, 324kB/s]
  0% 0.00/296 [00:00<?, ?B/s]
100% 296/296 [00:00<00:00, 282kB/s]
  0% 0.00/340 [00:00<?, ?B/s]
100% 340/340 [00:00<00:00, 328kB/s]
  0% 0.00/257 [00:00<?, ?B/s]
100% 257/257 [00:00<00:00, 223kB/s]
  0% 0.00/299 [00:00<?, ?B/s]
100% 299/299 [00:00<00:00, 295kB/s]
  0% 0.00/128 [00:00<?, ?B/s]
100% 128/128 [00:00<00:00, 127kB/s]
  0% 0.00/255 [00:00<?, ?B/s]
100% 255/255 [00:00<00:00, 279kB/s]
  0% 0.00/

In [0]:
pip install tensorflow==2.1.0-rc0

     |████████████████████████████████| 402.3MB 36kB/s 
     |████████████████████████████████| 3.8MB 30.7MB/s 
     |████████████████████████████████| 450kB 50.0MB/s 
ERROR: tensorflow-federated 0.12.0 has requirement tensorflow~=2.1.0, but you'll have tensorflow 2.1.0rc0 which is incompatible.
ERROR: tensorflow-federated 0.12.0 has requirement tensorflow-addons~=0.7.0, but you'll have tensorflow-addons 0.8.3 which is incompatible.
  Found existing installation: tensorboard 1.15.0
    Uninstalling tensorboard-1.15.0:
      Successfully uninstalled tensorboard-1.15.0
  Found existing installation: tensorflow-estimator 1.15.1
    Uninstalling tensorflow-estimator-1.15.1:
      Successfully uninstalled tensorflow-estimator-1.15.1
  Found existing installation: tensorflow 1.15.0
    Uninstalling tensorflow-1.15.0:
      Successfully uninstalled tensorflow-1.15.0


In [0]:
pip install keras==2.3.1


     |████████████████████████████████| 378kB 2.8MB/s 
  Found existing installation: Keras 2.2.5
    Uninstalling Keras-2.2.5:
      Successfully uninstalled Keras-2.2.5


In [0]:
from tqdm import tqdm
import pandas as pd
import numpy as np
struc=pd.read_csv("structures.csv.zip")
from sklearn.preprocessing import StandardScaler,LabelEncoder
ss=StandardScaler()
struc[['x','y','z']]=ss.fit_transform(struc[['x','y','z']])

atomic_radius = {'H':0.43, 'C':0.82, 'N':0.80, 'O':0.78, 'F':0.76} # With fudge factor
atoms = struc['atom'].values
struc['atom']=LabelEncoder().fit_transform(struc['atom'])
atoms_rad = [atomic_radius[x] for x in tqdm(atoms)]
struc["rad"]=atoms_rad
vals=struc[["x","y","z","rad",'atom_index','atom']].values
from numba import jit
from statistics import mean
nms=struc.groupby(["molecule_name"]).size().cumsum().values
ln=len(nms)
a=np.zeros(ln+1,dtype=int)
a[1:]=nms

from math import sqrt

@jit
def atm(id):
    start=a[id]
    stop=a[id+1]
    locs=vals[start:stop,:]
    num_atoms=stop-start
    dmat=np.ones((num_atoms,29,6))
    dmat=dmat*-999
    dmat[:,:,5]=7
    for i in range(num_atoms):
        for en,j in enumerate(range(i, num_atoms)):
            d = sqrt((locs[i,0] - locs[j,0])**2 + (locs[i,1] - locs[j,1])**2 + (locs[i,2] - locs[j,2])**2)
            dmat[i,j,0] = d
            dmat[j,i,0] = d
            dmat[i,j,1] = locs[j,4]
            dmat[j,i,1] = locs[i,4]
            
            dmat[i,j,5] = locs[j,5]
            dmat[j,i,5] = locs[i,5]
            

            dmat[i,j,2] = locs[j,0]
            dmat[j,i,2] = locs[i,0]
            
            
            dmat[i,j,3] = locs[j,1]
            dmat[j,i,3] = locs[i,1]
            
            
            dmat[i,j,4] = locs[j,2]
            dmat[j,i,4] = locs[i,2]
            
    return dmat
from tqdm import tqdm
def fnal(df):
    count=0
    ret=np.zeros((df.shape[0],29,6))
    for i in tqdm(range(len(df["molecule_name"].unique()))):
        dmat=atm(i)
        s=a[i+1]-a[i]
        for j in range(s):
            dmat[j,:,:]=dmat[j,np.argsort(dmat[j,:,0]),:]
        ret[a[i]:a[i+1],:,:]=dmat
    return ret

retu=fnal(struc)
struc['index']=struc.index
trn=pd.read_csv('train.csv.zip')
trn=trn.merge(struc[['molecule_name','atom_index','index','atom']],left_on=['atom_index_0','molecule_name'],right_on=['atom_index','molecule_name'])
trn=trn.merge(struc[['molecule_name','atom_index','index','atom']],left_on=['atom_index_1','molecule_name'],right_on=['atom_index','molecule_name'])
trn=trn.drop(['atom_index_x','atom_index_y'],1)

tst=pd.read_csv('test.csv.zip')
tst=tst.merge(struc[['molecule_name','atom_index','index']],left_on=['atom_index_0','molecule_name'],right_on=['atom_index','molecule_name'])
tst=tst.merge(struc[['molecule_name','atom_index','index']],left_on=['atom_index_1','molecule_name'],right_on=['atom_index','molecule_name'])
tst=tst.drop(['atom_index_x','atom_index_y'],1)
le=LabelEncoder()
trn['type']=le.fit_transform(trn['type'])
tst['type']=le.transform(tst['type'])


100%|██████████| 2358875/2358875 [00:00<00:00, 3179180.57it/s]


100%|██████████| 130789/130789 [00:20<00:00, 6238.07it/s]


In [0]:

import random, os, sys
import numpy as np
from keras.utils import Sequence
from keras.models import *
import os
import time
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm import tqdm
from math import ceil
from sklearn.model_selection import train_test_split
from sklearn import metrics
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, CuDNNLSTM, Embedding, Dropout, Activation, CuDNNGRU, Conv1D
from keras.layers import Bidirectional, GlobalMaxPool1D, GlobalMaxPooling1D, GlobalAveragePooling1D
from keras.layers import Input, Embedding, Dense, Conv2D, MaxPool2D, concatenate
from keras.layers import Reshape, Flatten, Concatenate, Dropout, SpatialDropout1D
from keras.layers import BatchNormalization, InputSpec, add
from keras.optimizers import Adam
from keras.models import Model, load_model
from keras import backend as K
from keras.engine.topology import Layer
from keras import initializers, regularizers, constraints, optimizers, layers, activations
from keras.callbacks import Callback, EarlyStopping, ModelCheckpoint,ReduceLROnPlateau
from keras.utils import Sequence
from keras.layers import *
from keras.callbacks import *
from keras.initializers import *
import tensorflow as tf
from keras.engine.topology import Layer

try:
    from dataloader import TokenList, pad_to_longest
    # for transformer
except: pass

class LayerNormalization(Layer):
    def __init__(self,  **kwargs):
        super(LayerNormalization, self).__init__(**kwargs)
    def build(self, input_shape):
        self.gamma = self.add_weight(name='gamma', shape=input_shape[-1:],
                                     initializer=Ones(), trainable=True)
        self.beta = self.add_weight(name='beta', shape=input_shape[-1:],
                                    initializer=Zeros(), trainable=True)
        super(LayerNormalization, self).build(input_shape)
    def call(self, x):
        mean = K.mean(x, axis=-1, keepdims=True)
        std = K.std(x, axis=-1, keepdims=True)
        return self.gamma * (x - mean) / (std + 1e-6) + self.beta
    def compute_output_shape(self, input_shape):
        return input_shape

class ScaledDotProductAttention():
    def __init__(self, d_model, attn_dropout=0.1):
        self.temper = np.sqrt(d_model)
        self.dropout = Dropout(attn_dropout)
    def __call__(self, q, k, v, mask):
        attn = Lambda(lambda x:K.batch_dot(x[0],x[1],axes=[2,2])/self.temper)([q, k])
        attn = Activation('softmax')(attn)
        attn = self.dropout(attn)
        output = Lambda(lambda x:K.batch_dot(x[0], x[1]))([attn, v])
        return output, attn

class MultiHeadAttention():
    # mode 0 - big martixes, faster; mode 1 - more clear implementation
    def __init__(self, n_head, d_model, d_k, d_v, dropout, mode=0, use_norm=True):
        self.mode = mode
        self.n_head = n_head
        self.d_k = d_k
        self.d_v = d_v
        self.dropout = dropout
        self.qs_layers = []
        self.ks_layers = []
        self.vs_layers = []
        for _ in range(n_head):
            self.qs_layers.append(TimeDistributed(Dense(100, use_bias=False,activation='relu')))
            self.ks_layers.append(TimeDistributed(Dense(100, use_bias=False,activation='relu')))
            self.vs_layers.append(TimeDistributed(Dense(100, use_bias=False,activation='relu')))
        self.attention = ScaledDotProductAttention(d_model)
        self.layer_norm = LayerNormalization() if use_norm else None
        self.w_o = TimeDistributed(Dense(522,activation='relu'))

    def __call__(self, q, k, v, mask=None):
        d_k, d_v = self.d_k, self.d_v
        n_head = self.n_head
        heads = []; attns = []
        for i in range(n_head):
            qs = self.qs_layers[i](q)   
            ks = self.ks_layers[i](k) 
            vs = self.vs_layers[i](v) 
            head, attn = self.attention(qs, ks, vs, mask)
            heads.append(head)
            attns.append(attn)
        head = Concatenate()(heads) if n_head > 1 else heads[0]
        attn = Concatenate()(attns) if n_head > 1 else attns[0]
        outputs = self.w_o(head)
        outputs = Dropout(self.dropout)(outputs)
        if not self.layer_norm: return outputs, attn
        outputs = Add()([outputs, q])
        return self.layer_norm(outputs), attn

class PositionwiseFeedForward():
    def __init__(self, d_hid, d_inner_hid, dropout=0.1):
        self.w_1 = Conv1D(d_inner_hid, 1, activation='relu',padding='same')
        self.w_2 = Conv1D(d_hid, 1,padding='same',activation='relu')
        self.layer_norm = LayerNormalization()
        self.dropout = Dropout(dropout)
    def __call__(self, x):
        output = self.w_1(x) 
        output = self.w_2(output)
        output = self.dropout(output)
        output = Add()([output, x])
        return self.layer_norm(output)

class EncoderLayer():
    def __init__(self, d_model, d_inner_hid, n_head, d_k, d_v, dropout=0.1):
        self.self_att_layer = MultiHeadAttention(n_head, d_model, d_k, d_v, dropout=dropout)
        self.pos_ffn_layer  = PositionwiseFeedForward(d_model, d_inner_hid, dropout=dropout)
    def __call__(self, enc_input, mask=None):
        output, slf_attn = self.self_att_layer(enc_input, enc_input, enc_input, mask=mask)
        #output = self.pos_ffn_layer(output)
        return output

class Encoder():
    def __init__(self, d_model, d_inner_hid, n_head, d_k, d_v, \
                layers=6, dropout=0.1, word_emb=None, pos_emb=None):
        self.emb_layer = word_emb
        self.emb_dropout = Dropout(dropout)
        self.layers = EncoderLayer(d_model, d_inner_hid, n_head, d_k, d_v, dropout) 
        
    def __call__(self, src_seq, return_att=False, active_layers=999):
            
        x = self.emb_dropout(src_seq)
        x = self.layers(x)
        x = self.layers(x)
        x = self.layers(x)
        x = self.layers(x)
        
        return x
class Transformer():
    def __init__(self, d_model=embed_size, \
              d_inner_hid=512, n_head=10, d_k=100, d_v=100, layers=2, dropout=0.1, \
              share_word_emb=False, **kwargs):
        self.name = 'Transformer'
        self.src_loc_info = True
        self.d_model = d_model
        self.decode_model = None
        d_emb = d_model

        self.encoder = Encoder(d_model, d_inner_hid, n_head, d_k, d_v, layers, dropout)

        
    def get_pos_seq(self, x):
        mask = K.cast(K.not_equal(x, 0), 'int32')
        pos = K.cumsum(K.ones_like(x, 'int32'), 1)
        return pos * mask

    def compile(self, active_layers=999):
        inp = Input(shape=(29,10),name='inp_main')
        typ = Input(shape=(1,),name='input_type')
        inp_a1 = Input(shape=(29,),name='input_atom1')
        inp_a2 = Input(shape=(29,),name='input_atom2')
        emb_typ = Embedding(9,522)(typ)
        emb_atm1 = Embedding(7,256)(inp_a1)
        emb_atm2 = Embedding(7,256)(inp_a2)
        src_seq = Concatenate(axis=2)([inp,emb_atm1,emb_atm2])
        

        x = self.encoder(src_seq, active_layers=active_layers)
        x = Concatenate(axis=1)([x,emb_typ])
        
        x = GlobalAveragePooling1D()(x)
        outp = Dense(1)(x)

        self.model = Model(inputs=[inp,typ,inp_a1,inp_a2], outputs=outp)
        self.model.compile(loss='mean_absolute_error', optimizer='adam')

class mape(Sequence):
    def __init__(self,data,df,batch_size,is_tst):
        self.map=data
        self.df=df
        self.batch_size=batch_size
        self.len=len(df)
        self.ck=is_tst
    def __len__(self):
        return int(ceil(len(self.df)/self.batch_size))
    def __getitem__(self,index):
        start=index*self.batch_size
        stop=min((index+1)*self.batch_size,self.len)
        inps=np.concatenate((self.map[self.df.loc[start:stop]['index_x'],:,:5],self.map[self.df.loc[start:stop]['index_y'],:,:5]),-1)
        atm1 = self.map[self.df.loc[start:stop]['index_x'],:,5]
        atm2 = self.map[self.df.loc[start:stop]['index_y'],:,5]
        if self.ck==0:
            outs=self.df.loc[start:stop]['scalar_coupling_constant'].reset_index(drop=True)
            return [inps,self.df.loc[start:stop]['type'].values,atm1,atm2],outs
        else:
            return [inps,self.df.loc[start:stop]['type'].values,atm1,atm2]
s2s = Transformer(maxlen, layers=1)



In [0]:
from keras.callbacks import ModelCheckpoint
from google.colab import drive
drive.mount('/content/gdrive')
drive.mount("/content/gdrive", force_remount=True)
path = F"/content/gdrive/My Drive/molecule_transform.hdf5" 
checkpoint = ModelCheckpoint(path, monitor='loss', verbose=1,save_best_only=True, mode='auto', period=1)

s2s.compile()
model = s2s.model
rlr = ReduceLROnPlateau(monitor='loss', factor=0.1,patience=2, min_lr=1e-6, mode='auto', verbose=1)



Using TensorFlow backend.


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
Mounted at /content/gdrive


In [0]:
model.fit_generator(mape(retu,trn,128,0),epochs=20,callbacks=[rlr,checkpoint])
pre=model.predict_generator(mape(retu,tst,128,1))
path = F"/content/gdrive/My Drive/weights_molecule.hdf5" 
model.save_weights(path)
ss=pd.read_csv('sample_submission.csv.zip')
ss['scalar_coupling_constant']=pre
ss=ss.set_index('id')
ss.to_csv('sub.csv')

Epoch 1/20
36400/36400 [==============================] - 3154s 87ms/step - loss: 3.9401

Epoch 00001: loss improved from 4.77374 to 3.94018, saving model to /content/gdrive/My Drive/molecule_transform.hdf5


TypeError: ignored